In [103]:
from fredapi import Fred

api_key_fred = os.environ['api_key_fred']
fred = Fred(api_key=api_key_fred)

In [104]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import plotly.graph_objects as go
import datetime
import pandas as pd
import csv
import os
import time
from dateutil import tz
import glob
import quandl as q
import re
import plotly.graph_objects as go
import numpy as np

quandl_api_key = os.environ['quandl_api_key']

# # # start - read in BTC data # # #
datasource_btcusd = "BITFINEX/BTCUSD.csv"
btcusd_data = pd.read_csv("coindata/{}".format(
    datasource_btcusd.replace("/", " ")), index_col=0)
btcusd_data.index = pd.to_datetime(btcusd_data.index)

most_recent_stored_btcusd_date = btcusd_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")
todays_date = datetime.date.today() - timedelta(days=1)
todays_date = todays_date.strftime("%Y-%m-%d")


if most_recent_stored_btcusd_date != todays_date:
    # start - get data from quandl api
    data = q.get(datasource_btcusd.split(".")[0],   start_date=most_recent_stored_btcusd_date,
                    end_date='{}'.format(todays_date),
                    api_key=quandl_api_key)
    data.info()
    data["First"] = data.Last.shift(1)
    data.dropna()
    btcusd_data = pd.concat([btcusd_data, data])
    btcusd_data = btcusd_data.sort_index()
    # store current df with up-to-date values
    btcusd_data.to_csv('coindata/{}'.format(
        datasource_btcusd.replace("/", " ")), index=True)
        
# # # end - read in BTC data # # #

In [105]:
fed_assets_quandl_key = "FED/RESPPA_N_WW"
fed_assets_data = q.get(fed_assets_quandl_key, 
            api_key=quandl_api_key
            )
fed_assets_data = fed_assets_data.dropna()
fed_assets_data.info()
fed_assets_data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1039 entries, 2002-12-18 to 2022-11-09
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Value   1039 non-null   float64
dtypes: float64(1)
memory usage: 16.2 KB


,Value
Date,
2002-12-18,720761.00
2002-12-25,733136.00
2003-01-01,732202.00
2003-01-08,724902.00
2003-01-15,721325.00
...,...
2022-10-12,8758969.00
2022-10-19,8743922.00
2022-10-26,8723090.00


In [106]:
fed_assets_data = fed_assets_data.sort_index()
fed_assets_data

,Value
Date,
2002-12-18,720761.00
2002-12-25,733136.00
2003-01-01,732202.00
2003-01-08,724902.00
2003-01-15,721325.00
...,...
2022-10-12,8758969.00
2022-10-19,8743922.00
2022-10-26,8723090.00


In [107]:
# store current df with up-to-date values
fed_assets_data.to_csv('coindata/{}'.format(
    fed_assets_quandl_key.replace("/", " ")), index=True)

In [108]:
# merge fed and btc
btcusd_data_and_fed = pd.merge(btcusd_data, fed_assets_data, left_index=True, right_index=True, how='left')
btcusd_data_and_fed

,High,Low,Mid,Last,Bid,Ask,Volume,First,Value
Date,,,,,,,,,
2014-04-15,513.90,452.00,504.24,505.00,503.50,504.97,21013.58,NaN,NaN
2014-04-16,547.00,495.00,537.50,538.00,537.00,538.00,29633.36,505.00,4283967.00
2014-04-17,538.50,486.10,507.02,508.00,506.04,508.00,20709.78,538.00,NaN
2014-04-18,509.00,474.25,483.77,482.75,482.75,484.79,10458.05,508.00,NaN
2014-04-19,513.99,473.83,505.01,507.50,502.53,507.49,8963.62,482.75,NaN
...,...,...,...,...,...,...,...,...,...
2022-11-14,17200.00,15822.00,16669.50,16689.00,16668.00,16671.00,6652.68,16343.00,NaN
2022-11-14,17200.00,15822.00,16669.50,16689.00,16668.00,16671.00,6652.68,NaN,NaN
2022-11-15,17132.00,16543.00,16865.50,16868.00,16865.00,16866.00,6520.96,16689.00,NaN


In [109]:
btcusd_data_and_fed["Value"] = btcusd_data_and_fed["Value"].ffill()
btcusd_data_and_fed = btcusd_data_and_fed.dropna()

In [110]:
btcusd_data_and_fed["BTC_per_FedAssets"] = (btcusd_data_and_fed["Last"] / btcusd_data_and_fed["Value"]) *1000
btcusd_data_and_fed["350_movingaverage_per_FedAssets"] = pd.Series.rolling(btcusd_data_and_fed["BTC_per_FedAssets"], window=350, min_periods=1).mean()
btcusd_data_and_fed["111_movingaverage_per_FedAssets"] = pd.Series.rolling(btcusd_data_and_fed["BTC_per_FedAssets"], window=111, min_periods=1).mean()

C:\Users\gebel\AppData\Local\Temp\ipykernel_8488\2281456468.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\gebel\AppData\Local\Temp\ipykernel_8488\2281456468.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\gebel\AppData\Local\Temp\ipykernel_8488\2281456468.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

# GET S&P DATA

In [111]:
# Billions of Dollars 
data_SP500 = fred.get_series('SP500')
data_SP500 = data_SP500 * 1000

data_SP500.dropna()
data_SP500 = data_SP500.sort_index()
# store current df with up-to-date values
data_SP500.to_csv('coindata/data_SP500.csv', index=True)

print(data_SP500.head(1))
print(data_SP500.tail(1))

2012-11-19   1386890.00
dtype: float64
2022-11-17   3946560.00
dtype: float64


# Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW)

In [112]:
start_date='2014-04-21'

In [113]:
#Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW) Millions of Dollars
data_WALCL = fred.get_series('WALCL')

data_WALCL.dropna()
data_WALCL = data_WALCL.sort_index()
# store current df with up-to-date values
data_WALCL.to_csv('coindata/data_WALCL.csv', index=True)

print(data_WALCL.head(1))
print(data_WALCL.tail(1))

2002-12-18   719542.00
dtype: float64
2022-11-16   8625620.00
dtype: float64


# Overnight Reverse Repurchase Agreements Treasury Securities Sold by the Federal Reserve in the Temporary Open Market Operations (RRPONTSYD)

In [114]:
data_RRPONTSYD = fred.get_series('RRPONTSYD')
data_RRPONTSYD

2003-02-07      2.50
2003-02-10       NaN
2003-02-11       NaN
2003-02-12       NaN
2003-02-13       NaN
               ...  
2022-11-11       NaN
2022-11-14   2165.02
2022-11-15   2086.57
2022-11-16   2099.07
2022-11-17   2114.44
Length: 5160, dtype: float64

In [115]:
#Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW)
# Billions of U.S. Dollars
# data_FRED_RRPONTSYD.values = data_FRED_RRPONTSYD.values * 1000 
data_RRPONTSYD = data_RRPONTSYD * 1000 
data_RRPONTSYD.dropna()
data_RRPONTSYD = data_RRPONTSYD.sort_index()
# store current df with up-to-date values
data_RRPONTSYD.to_csv('coindata/data_RRPONTSYD.csv', index=True)

print(data_RRPONTSYD.head(1))
print(data_RRPONTSYD.tail(1))

2003-02-07   2500.00
dtype: float64
2022-11-17   2114439.00
dtype: float64


# Deposits with Federal Reserve Banks, other than Reserve Balances: U.S. Treasury, General Account (WTREGEN)

In [116]:
# Billions of Dollars 
data_FRED_WTREGEN = fred.get_series('WTREGEN')
data_FRED_WTREGEN = data_FRED_WTREGEN * 1000

data_FRED_WTREGEN.dropna()
data_FRED_WTREGEN = data_FRED_WTREGEN.sort_index()
# store current df with up-to-date values
data_FRED_WTREGEN.to_csv('coindata/data_FRED_WTREGEN.csv', index=True)

print(data_FRED_WTREGEN.head(1))
print(data_FRED_WTREGEN.tail(1))

1986-01-08   4140.00
dtype: float64
2022-11-16   502982.00
dtype: float64


# net liquidity vs s&p500 weekly - Diagram

In [117]:
datasource_fred_total_assets = "data_WALCL.csv"
fred_total_assets = pd.read_csv("coindata/{}".format(
    datasource_fred_total_assets.replace("/", " ")), index_col=0)
fred_total_assets.index = pd.to_datetime(fred_total_assets.index)

most_recent_stored_fred_rrpontsyd_date = fred_total_assets.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")
todays_date = datetime.date.today() - timedelta(days=1)
todays_date = todays_date.strftime("%Y-%m-%d")
fred_total_assets

,0
2002-12-18,719542.00
2002-12-25,732059.00
2003-01-01,730994.00
2003-01-08,723762.00
2003-01-15,720074.00
...,...
2022-10-19,8743922.00
2022-10-26,8723090.00
2022-11-02,8676870.00
2022-11-09,8678886.00


In [118]:
datasource_FRED_WTREGEN = "data_FRED_WTREGEN.csv"
FRED_WTREGEN_data = pd.read_csv("coindata/{}".format(
    datasource_FRED_WTREGEN.replace("/", " ")), index_col=0)
FRED_WTREGEN_data.index = pd.to_datetime(FRED_WTREGEN_data.index)

most_recent_stored_FRED_WTREGEN_date = FRED_WTREGEN_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")
FRED_WTREGEN_data

,0
1986-01-08,4140.00
1986-01-15,5032.00
1986-01-22,13856.00
1986-01-29,17487.00
1986-02-05,12475.00
...,...
2022-10-19,607199.00
2022-10-26,634548.00
2022-11-02,598544.00
2022-11-09,527479.00


In [119]:
datasource_FRED_RRPONTSYD = "data_RRPONTSYD.csv"
FRED_RRPONTSYD_data = pd.read_csv("coindata/{}".format(
    datasource_FRED_RRPONTSYD.replace("/", " ")), index_col=0)
FRED_RRPONTSYD_data.index = pd.to_datetime(FRED_RRPONTSYD_data.index)

most_recent_stored_FRED_RRPONTSYD_date = FRED_RRPONTSYD_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")
FRED_RRPONTSYD_data

,0
2003-02-07,2500.00
2003-02-10,NaN
2003-02-11,NaN
2003-02-12,NaN
2003-02-13,NaN
...,...
2022-11-11,NaN
2022-11-14,2165018.00
2022-11-15,2086574.00
2022-11-16,2099070.00


In [120]:
type( FRED_RRPONTSYD_data.iloc[0].name )

pandas._libs.tslibs.timestamps.Timestamp

# filter for datetime to allign all fred data sources!

In [121]:
merged_FRED_RRPONTSYD_data = pd.merge(FRED_WTREGEN_data,FRED_RRPONTSYD_data, how='left', left_index=True, right_index=True)
# merged.isnull().sum()
del merged_FRED_RRPONTSYD_data["0_x"]
merged_FRED_RRPONTSYD_data.columns=["0"]

In [122]:
netLiquidity = fred_total_assets-merged_FRED_RRPONTSYD_data-FRED_WTREGEN_data

In [123]:
print(len(merged_FRED_RRPONTSYD_data))
print(len(fred_total_assets))
print(len(FRED_WTREGEN_data))

1924
1040
1924


In [124]:
# df['dt'] = pd.to_datetime(df['dt'])
FRED_RRPONTSYD_data = FRED_RRPONTSYD_data[(FRED_RRPONTSYD_data.index > '2010-08-11 00:00:00')]
fred_total_assets = fred_total_assets[(fred_total_assets.index > '2010-08-11 00:00:00')]
FRED_WTREGEN_data = FRED_WTREGEN_data[(FRED_WTREGEN_data.index > '2010-08-11 00:00:00')]

fred_total_assets.head()
fred_total_assets.columns = ['Value_fred_total_assets']
len(fred_total_assets)

FRED_WTREGEN_data.head()
FRED_WTREGEN_data.columns = ['Value_FRED_WTREGEN_data']
len(FRED_WTREGEN_data)

FRED_RRPONTSYD_data.head()
FRED_RRPONTSYD_data.columns = ['Value_FRED_RRPONTSYD_data']
len(FRED_RRPONTSYD_data)

# bring dfs together and forwardfill all!
frames = [FRED_RRPONTSYD_data, FRED_WTREGEN_data, fred_total_assets]
result = pd.concat(frames)

In [125]:
# result["netLIQ"] = result['Value_FRED_RRPONTSYD_data'] - result['Value_FRED_WTREGEN_data'] - result['Value_fred_total_assets'] 

In [126]:
# result["netLIQ"] = result['Value_FRED_RRPONTSYD_data'] - result['Value_FRED_WTREGEN_data'] - result['Value_fred_total_assets'] 

In [127]:
data_SP500

2012-11-19   1386890.00
2012-11-20   1387810.00
2012-11-21   1391030.00
2012-11-22          NaN
2012-11-23   1409150.00
                ...    
2022-11-11   3992930.00
2022-11-14   3957250.00
2022-11-15   3991730.00
2022-11-16   3958790.00
2022-11-17   3946560.00
Length: 2609, dtype: float64

In [128]:
# data_SP500_2weeksback = data_SP500.shift(-2,"W")
# data_SP500_2weeksback

2012-11-11   1386890.00
2012-11-11   1387810.00
2012-11-11   1391030.00
2012-11-11          NaN
2012-11-11   1409150.00
                ...    
2022-10-30   3992930.00
2022-11-06   3957250.00
2022-11-06   3991730.00
2022-11-06   3958790.00
2022-11-06   3946560.00
Length: 2609, dtype: float64

In [129]:
data_SP500_1weekback = data_SP500.shift(-1, "W")
data_SP500_1weekback

2012-11-18   1386890.00
2012-11-18   1387810.00
2012-11-18   1391030.00
2012-11-18          NaN
2012-11-18   1409150.00
                ...    
2022-11-06   3992930.00
2022-11-13   3957250.00
2022-11-13   3991730.00
2022-11-13   3958790.00
2022-11-13   3946560.00
Length: 2609, dtype: float64

In [130]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

In [131]:
netLiquidity = netLiquidity[(netLiquidity.index > '2012-11-18 00:00:00')]
netLiquidity = netLiquidity.squeeze()
netLiquidity = netLiquidity.dropna()
netLiquidity

2013-04-03   3139168.00
2013-08-07   3522254.00
2013-08-14   3580992.00
2013-09-25   3682783.00
2013-10-02   3706724.00
                ...    
2022-10-19   5894888.00
2022-10-26   5901686.00
2022-11-02   5848465.00
2022-11-09   5913595.00
2022-11-16   6023568.00
Name: 0, Length: 473, dtype: float64

In [132]:
data_SP500_1weekback = data_SP500_1weekback[(data_SP500_1weekback.index > '2013-08-12 00:00:00')]
# data_SP500_1weekback = data_SP500_1weekback.index.drop_duplicates()
data_SP500_1weekback = data_SP500_1weekback[~data_SP500_1weekback.index.duplicated(keep='first')]
data_SP500_1weekback = data_SP500_1weekback.dropna()
data_SP500_1weekback

2013-08-18   1646060.00
2013-08-25   1656780.00
2013-09-08   1671710.00
2013-09-15   1697600.00
2013-09-22   1701840.00
                ...    
2022-10-16   3677950.00
2022-10-23   3797340.00
2022-10-30   3871980.00
2022-11-06   3806800.00
2022-11-13   3957250.00
Length: 438, dtype: float64

In [ ]:
data_SP500_2weeksback = data_SP500_1weekback.shift(-1, "W")
data_SP500_2weeksback

In [133]:
dfdiffsp500_netliq = pd.concat([netLiquidity, data_SP500_1weekback], axis=1, keys=('netLiquidity','data_SP500_1weekback'), join='outer').ffill(axis = 0).dropna()
dfdiffsp500_netliq["netLiquidity"]  = dfdiffsp500_netliq["netLiquidity"] / 1.1 - 1625

dfdiffsp500_netliq["diff"] = dfdiffsp500_netliq["data_SP500_1weekback"]  - dfdiffsp500_netliq["netLiquidity"]  

In [142]:
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(
    go.Scatter(
        x=dfdiffsp500_netliq.index,
        y=dfdiffsp500_netliq["diff"],
        name="diffsp500_netliq",
        mode='lines',
        marker=dict(
            # size=16,
            color="red",  # set color equal to a variable
            # colorscale='Viridis', # one of plotly colorscales
            # showscale=True
        ), 
    ), 
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(
        x=data_SP500_2weeksback.index,
        y=data_SP500_2weeksback.values,
        name="SP500 - 2 weeks ago",
        mode='lines',
        marker=dict(
            # size=16,
            color="blue",  # set color equal to a variable
            # colorscale='Viridis', # one of plotly colorscales
            # showscale=True
        ), 
    ), 
    secondary_y=True,
)


fig.add_trace(
    go.Scatter(
        x=data_SP500_1weekback.index,
        y=data_SP500_1weekback.values,
        name="SP500 - 1 week ago",
        mode='lines',
        marker=dict(
            # size=16,
            color="blue",  # set color equal to a variable
            # colorscale='Viridis', # one of plotly colorscales
            # showscale=True
        ), 
    ), 
    secondary_y=True,
)

fig.add_trace(go.Scatter(x=fred_total_assets.index, y=fred_total_assets["0"],
                    mode='lines',
                    name='fred_total_assets',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="black"
                        ),
                    )
                )

fig.update_layout(yaxis=dict(domain=[0, 0.7]) )

fig.add_trace(go.Scatter(x=netLiquidity.index, y=netLiquidity.values, stackgroup='one', #fill='tonexty',
                    mode='lines',
                    name='netLiquidity',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="green"
                        )
                    )
                )

fig.add_trace(go.Scatter(x=merged_FRED_RRPONTSYD_data.index, y=merged_FRED_RRPONTSYD_data["0"], stackgroup='one', #fill='tonexty',
                    mode='lines',
                    name='merged_FRED_RRPONTSYD_data',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="yellow"
                        )
                    )
                )

fig.add_trace(go.Scatter(x=FRED_WTREGEN_data.index, y=FRED_WTREGEN_data["0"], stackgroup='one', #fill='tonexty',
                    mode='lines',
                    name='FRED_WTREGEN_data',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="red"
                        ),
                    )
                )

                
# btcusd_data["111_movingaverage"]
# fig.update_yaxes(type="log") #, range=[0,5]
fig.add_shape(type='line',
                x0=FRED_WTREGEN_data.index[0],
                y0=1250000,
                x1=FRED_WTREGEN_data.index[-1],
                y1=1250000,
                line=dict(color='Red',),
                xref='x',
                yref='y'
)
fig.add_shape(type='line',
                x0=FRED_WTREGEN_data.index[0],
                y0=300000,
                x1=FRED_WTREGEN_data.index[-1],
                y1=300000,
                line=dict(color='Red',),
                xref='x',
                yref='y'
)

fig.update_layout(
    # title="Plot Title",
    autosize=False,
    width=int(1400/1.3),
    height=int(800/1.3),
)
fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)
fig

In [135]:
aaaaasdasd

NameError: name 'aaaaasdasd' is not defined

In [ ]:
import plotly.express as px
df = px.data.gapminder()
fig = px.area(df, x="year", y="pop", color="continent", line_group="country")
fig.show()

In [ ]:
#Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW)
fed_assets_quandl_key = "FED/RESPPA_N_WW"

fed_assets_data = pd.read_csv("coindata/{}".format(
    fed_assets_quandl_key.replace("/", " ")), index_col=0)
fed_assets_data.index = pd.to_datetime(fed_assets_data.index)

In [ ]:
fed_assets_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")

In [ ]:
from datetime import datetime, timedelta
import datetime
most_recent_stored_fed_assets_date = fed_assets_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")

todays_date = datetime.date.today() - timedelta(days=7)
todays_date = todays_date.strftime("%Y-%m-%d")


In [ ]:
most_recent_stored_fed_assets_date

In [ ]:
todays_date

In [ ]:
fig = go.Figure(
    data=go.Scatter(
        x=btcusd_data_and_fed.index,
        y=btcusd_data_and_fed['BTC_per_FedAssets'],
        name="BTC/FED",
        mode='lines',
        marker=dict(
            # size=16,
            color="red",  # set color equal to a variable
            # colorscale='Viridis', # one of plotly colorscales
            # showscale=True
        )
)
)

fig.add_trace(go.Scatter(x=btcusd_data_and_fed.index, y=btcusd_data_and_fed["350_movingaverage_per_FedAssets"],
                    mode='lines',
                    name='350_movingaverage_per_FedAssets',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="gold"
                        ),
                    )
                )

fig.add_trace(go.Scatter(x=btcusd_data_and_fed.index, y=btcusd_data_and_fed["350_movingaverage_per_FedAssets"]*2,
                    mode='lines',
                    name='2*350_movingaverage_per_FedAssets',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="red"
                        ),
                    )
                )

fig.add_trace(go.Scatter(x=btcusd_data.index, y=btcusd_data_and_fed["111_movingaverage_per_FedAssets"],
                    mode='lines',
                    name='111_movingaverage_per_FedAssets',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="green"
                        )
                    )
                )

# btcusd_data["111_movingaverage"]
fig.update_yaxes(type="log") #, range=[0,5]

fig.update_layout(
    # title="Plot Title",
    autosize=False,
    width=int(1400/1.3),
    height=int(800/1.3),
)

fig

In [ ]:
btcusd_data = btcusd_data.dropna()
btcusd_data["350_movingaverage"] = pd.Series.rolling(btcusd_data["Last"], window=350, min_periods=1).mean()
btcusd_data["111_movingaverage"] = pd.Series.rolling(btcusd_data["Last"], window=111, min_periods=1).mean()
btcusd_data

In [ ]:
fig = go.Figure(
    data=go.Scatter(
        x=btcusd_data.index,
        y=btcusd_data['Last'],
        mode='lines',
        marker=dict(
            # size=16,
            color="black",  # set color equal to a variable
            # colorscale='Viridis', # one of plotly colorscales
            # showscale=True
        )
    )
)

fig.add_trace(go.Scatter(x=btcusd_data.index, y=btcusd_data["350_movingaverage"]*2,
                    mode='lines',
                    name='350_movingaverage',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="red"
                        ),
                    )
                )

fig.add_trace(go.Scatter(x=btcusd_data.index, y=btcusd_data["111_movingaverage"],
                    mode='lines',
                    name='111_movingaverage',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="green"
                        )
                    )
                )

# btcusd_data["111_movingaverage"]
fig.update_yaxes(type="log") #, range=[0,5]

fig.update_layout(
    # title="Plot Title",
    autosize=False,
    width=int(1400/1.3),
    height=int(800/1.3),
)

fig